In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split as tts

from sklearn.preprocessing import StandardScaler, MinMaxScaler  
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2

from sklearn.linear_model import LinearRegression as LinReg
from sklearn.linear_model import Lasso        # regularizacion L1
from sklearn.linear_model import Ridge        # regularizacion L2
from sklearn.linear_model import ElasticNet

from sklearn.linear_model import LogisticRegression as LogReg

from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor as RFR  
from sklearn.tree import ExtraTreeRegressor as ETR
from sklearn.ensemble import GradientBoostingRegressor as GBR
from xgboost import XGBRegressor as XGBR
from lightgbm import LGBMRegressor as LGBMR
from sklearn.model_selection import GridSearchCV
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
from catboost import CatBoostRegressor as CTR
from lightgbm import LGBMRegressor as LGBMR
from sklearn.naive_bayes import GaussianNB

In [49]:
def ml1():
    
    models = {'RFR':{'MODEL':RFR(),'PARAM':{'n_estimators': [10, 50, 100, 150, 200, 500],'max_depth': [1, 5, 10, 15, 20],'min_weight_fraction_leaf':[0.0,0.1,0.2]}},
              'XGB':{'MODEL':XGBR(),'PARAM':{'n_estimators': [10, 50, 150, 200, 500,1000],'max_depth': [1, 5, 10, 15, 20],'learning_rate':[0.001,0.002,0.01,0.05] }},
              #'CTR':{'MODEL':CTR(),'PARAM:':{'depth' : [6,8,10],'learning_rate' : [0.01, 0.05, 0.1],'iterations': [30, 50, 100],'subsample' : [0.5, 0.7, 1.0]}},
              #'GaussianNB':{'MODEL':GaussianNB(),'PARAM':{'var_smoothing':[1e-09]}},
              'Lasso':{'MODEL':Lasso(),'PARAM':{'alpha':[0.3,0.5,0.7,0.9,1.0],'max_iter':[800,1000,1200]}},
              'LGBMR': {'MODEL':LGBMR(),'PARAM':{'boosting_type':['gbdt','dart']}}}
    
            
            
                     
    rmse = []
    name = []
    #b = []
    score = []
    dfmodels = pd.DataFrame()
    for m in models:
        x = models[m]["MODEL"]
        p = models[m]["PARAM"]
        y_pred=grid(x,p).predict(X_test)
        #best= grid(x,p).best_params_   
        #sco = grid(x,p).best_score_
        MSE = mse(y_test, y_pred, squared=False)
        #score.append(sco)
        #b.append(best)
        rmse.append(MSE)
        name.append(m)
    dfmodels['Modelo'] = name
    dfmodels['RMSE'] = rmse
    #dfmodels['Best_Parametres'] = b
    #dfmodels['bestsco'] = score                        
    dfmodels.sort_values("RMSE",ascending=True,inplace=True,ignore_index=True)
    print(f'model {dfmodels.Modelo[0]} rmse {dfmodels.RMSE[0]} ')
    return dfmodels

In [2]:
def grid(modelo, param):
    
    g=GridSearchCV(modelo, # modelo de sklearn
                   param,  # dictio de parametros
                   cv=5,   # nº de cortes del cross-validation
                   return_train_score=True, # error en entrenamiento para checkear
                   n_jobs=-1  # usa todos los nucleos disponibles
                  )

    g.fit(X_train, y_train)
    print('Acierto test: {:.2f}'.format(g.score(X_test, y_test)))
    print('Acierto train: {:.2f}'.format(g.score(X_train, y_train)))
    print('Mejores parametros: {}'.format(g.best_params_))
    print('Modelo: {}'.format(modelo))
    print('Mejor acierto cv: {:.2f}'.format(g.best_score_))

   

    return g.best_estimator_.fit(X_train, y_train)

In [3]:
salaries=pd.read_csv(r'data/salaries_data.csv')
testeo=pd.read_csv(r'data/testeo.csv')
muestra=pd.read_csv(r'data/muestra.csv')

In [4]:
average = {"US":54132,"FR":41379,"GR":17093,"LU": 75305,"SI":29460,"DE":45485,"IN":4766,"GB":40519,"PK":4379,"MD":6480,"JP":45000,"CA":40000,"AS":52626,"IE":40000,"AE":50000,"MX":20000,"VN":2400,"BE":44300,"KE":7608,"ES":36000,"CH":50000,"CL":27816,"CN":16716,"DK":40000,"RO":8360,"MY":18877,"PR":33000,"HU":17640,"PT": 31200,"BG":22263, "HN": 14000,"TR":8400,"AU": 52338,"UA":10000,"NG":9000,"BR":19000,"NL":50000,"HK":56000,"SG":60000,"IR":6000,"NZ":61000,"RS":16800,"JE":60000,"RU":18000,"EE":22000,"CZ":26000,"AT":45000,"BO":6000,"PL":23000,"IT":38000,"MT":30000,"DZ":7000,"HR":12000,"PH":13000,"IQ":5500,"CO":12000,"AR":20000,"TN":6000,"IL":42000}

In [5]:
jobs = {'Data Scientist':124000,'Data Engineer':111000,'Data Analyst':71599,'Data Architect':160000,'Machine Learning':138000,'Head of Data':170000,'ETL Developer':110000,'Data Specialist':160000,'3D Computer Vision Researcher':90000}

In [6]:
lst3 = []
lst4 = []
for i in jobs.keys():
    lst3.append(i)
    lst4.append(jobs[i])

In [7]:
lst1 = []
lst2 = []
for i in average.keys():
    lst1.append(i)
    lst2.append(average[i])

In [8]:
data1 = {"cc":lst1,"salary":lst2}

In [9]:
data2 = {"job":lst3,"salary":lst4}

In [10]:
salary = pd.DataFrame(data= data1)

In [11]:
salary2= pd.DataFrame(data= data2)

In [25]:
for i in salaries.job_title:
    if 'Engineer' in i:
        salaries.job_title.replace({i: 'Data Engineer'}, inplace = True)
    elif 'Analy' in i:
        salaries.job_title.replace({i: 'Data Analyst'}, inplace = True)
    elif 'Architect' in i:
        salaries.job_title.replace({i: 'Data Architect'}, inplace = True)
    elif 'Scien' in i:
        salaries.job_title.replace({i: 'Data Scientist'}, inplace = True)
    elif 'Machine'in i:
        salaries.job_title.replace({i: 'Machine Learning'}, inplace = True)

In [26]:
for i,e in enumerate(salaries.employee_residence):
    f = salary[salary.cc==e].salary
    f = int(f)
    salaries.employee_residence[i]=f

C:\Users\pabli\anaconda3\envs\clase\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [27]:
for i,e in enumerate(salaries.company_location):
    f = salary[salary.cc==e].salary
    f = int(f)
    salaries.company_location[i]=f

C:\Users\pabli\anaconda3\envs\clase\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [28]:
for i,e in enumerate(salaries.job_title):
    try:
        f = salary2[salary2.job==e].salary
        f = int(f)
        salaries.job_title[i]=f
    except:
        print(e)

C:\Users\pabli\anaconda3\envs\clase\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [30]:
numeric1 = ['work_year','employee_residence',"company_location",'job_title']

In [31]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

salaries[numeric1]=scaler.fit_transform(salaries[numeric1])

salaries.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0.857874,SE,FT,0.080224,140250,USD,140250,0.653275,100,0.586726,M
1,0.857874,SE,FT,0.080224,135000,USD,135000,0.653275,100,0.586726,M
2,-0.606075,MI,FT,-1.738256,100000,USD,100000,0.653275,100,0.586726,M
3,-0.606075,MI,CT,0.080224,270000,USD,270000,0.653275,100,0.586726,L
4,-0.606075,MI,FT,0.080224,22000,EUR,26005,-2.260379,0,0.586726,L


In [38]:
salaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   work_year           500 non-null    float64
 1   experience_level    500 non-null    object 
 2   employment_type     500 non-null    object 
 3   job_title           500 non-null    float64
 4   salary              500 non-null    int64  
 5   salary_currency     500 non-null    object 
 6   salary_in_usd       500 non-null    int64  
 7   employee_residence  500 non-null    float64
 8   remote_ratio        500 non-null    int64  
 9   company_location    500 non-null    float64
 10  company_size        500 non-null    object 
dtypes: float64(4), int64(3), object(4)
memory usage: 43.1+ KB


In [39]:
for e,i in enumerate(salaries.remote_ratio):
    if i == 100:
        salaries.remote_ratio[e]= "Full"
    elif i == 50:
        salaries.remote_ratio[e]= "Half"
    elif i == 0:
        salaries.remote_ratio[e]= "None"
        
    

C:\Users\pabli\anaconda3\envs\clase\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\pabli\anaconda3\envs\clase\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [40]:
salaries

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0.857874,SE,FT,0.080224,140250,USD,140250,0.653275,Full,0.586726,M
1,0.857874,SE,FT,0.080224,135000,USD,135000,0.653275,Full,0.586726,M
2,-0.606075,MI,FT,-1.738256,100000,USD,100000,0.653275,Full,0.586726,M
3,-0.606075,MI,CT,0.080224,270000,USD,270000,0.653275,Full,0.586726,L
4,-0.606075,MI,FT,0.080224,22000,EUR,26005,-2.260379,None,0.586726,L
...,...,...,...,...,...,...,...,...,...,...,...
495,0.857874,SE,FT,0.080224,100000,USD,100000,0.653275,Full,0.586726,M
496,-0.606075,EN,FT,0.080224,435000,INR,5882,-2.489158,None,0.293828,L
497,0.857874,SE,FT,2.341728,90700,USD,90700,-0.246309,Full,-0.415023,M
498,-0.606075,MI,FT,0.080224,60000,GBP,82528,-0.213271,Full,-0.378233,L


In [41]:
salaries=pd.get_dummies(salaries, columns=['company_size','employment_type','remote_ratio','experience_level'], drop_first=True)

salaries.head()

,work_year,job_title,salary,salary_currency,salary_in_usd,employee_residence,company_location,company_size_M,company_size_S,employment_type_FL,employment_type_FT,employment_type_PT,remote_ratio_Half,remote_ratio_None,experience_level_EX,experience_level_MI,experience_level_SE
0,0.857874,0.080224,140250,USD,140250,0.653275,0.586726,1,0,0,1,0,0,0,0,0,1
1,0.857874,0.080224,135000,USD,135000,0.653275,0.586726,1,0,0,1,0,0,0,0,0,1
2,-0.606075,-1.738256,100000,USD,100000,0.653275,0.586726,1,0,0,1,0,0,0,0,1,0
3,-0.606075,0.080224,270000,USD,270000,0.653275,0.586726,0,0,0,0,0,0,0,0,1,0
4,-0.606075,0.080224,22000,EUR,26005,-2.260379,0.586726,0,0,0,1,0,0,1,0,1,0


In [42]:
salaries.drop(columns=['salary','salary_currency'],inplace=True)


In [43]:
X = salaries.drop('salary_in_usd', axis=1)

y = salaries.salary_in_usd

X.head()

,work_year,job_title,employee_residence,company_location,company_size_M,company_size_S,employment_type_FL,employment_type_FT,employment_type_PT,remote_ratio_Half,remote_ratio_None,experience_level_EX,experience_level_MI,experience_level_SE
0,0.857874,0.080224,0.653275,0.586726,1,0,0,1,0,0,0,0,0,1
1,0.857874,0.080224,0.653275,0.586726,1,0,0,1,0,0,0,0,0,1
2,-0.606075,-1.738256,0.653275,0.586726,1,0,0,1,0,0,0,0,1,0
3,-0.606075,0.080224,0.653275,0.586726,0,0,0,0,0,0,0,0,1,0
4,-0.606075,0.080224,-2.260379,0.586726,0,0,0,1,0,0,1,0,1,0


In [45]:
testeo.employment_type.value_counts()

FT    104
PT      2
CT      1
Name: employment_type, dtype: int64

In [46]:
X.drop(columns="employment_type_FL",inplace=True)

In [47]:
from sklearn.model_selection import train_test_split as tts

X_train, X_test, y_train, y_test = tts(X, y, train_size=0.8, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((400, 13), (100, 13), (400,), (100,))

In [51]:
ml1()

Acierto test: 0.33
Acierto train: 0.64
Mejores parametros: {'max_depth': 5, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 150}
Modelo: RandomForestRegressor()
Mejor acierto cv: 0.48
Acierto test: 0.40
Acierto train: 0.55
Mejores parametros: {'learning_rate': 0.05, 'max_depth': 1, 'n_estimators': 500}
Modelo: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=None,
             gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=None,
        

,Modelo,RMSE
0,XGB,48572.030168
1,LGBMR,49715.507009
2,RFR,50576.830072
3,Lasso,51298.447306
